# Домашнее задание № 4. Языковые модели

## Задание 1

In [2]:
import numpy as np
from collections import Counter
from nltk import sent_tokenize
from razdel import tokenize
from string import punctuation


def filter_text(text: str) -> list:
    """Убираем лишние пробелы и переводы строк"""
    return [i.text for i in tokenize(text.strip().replace('\n', ''))]


def get_ngrams(sent: list, length: int) -> list:
    if length == 1:
        return sent
    result = []
    for i, word in enumerate(sent[:-length + 1]):
        result.append(tuple(sent[i:i + length]))
    return result


def glue_punctuation(text: str) -> str:
    """Удаляем пробелы перед знаками препинания"""
    space_before_punkt = [f' {i}' for i in punctuation + '»']
    for i in space_before_punkt:
        text = text.replace(i, i[1])
    text = text.replace('« ', '«') # открывающаяся кавычка, наоборот "приклеивается" к последующему тексту
    return text


def generate_sents(probs: dict, num_sents: int, start=('<start>', '<start>')):
    for _ in range(num_sents):
        sent = start
        current_bigram = start
        next_word = None
        while next_word != '<end>':
            candidate_words, word_probs = zip(
                *{tri[2]: probs[tri] for tri in probs if tri[:2] == current_bigram}.items())
            # Из словаря триграмм выбираем те, что начинаются с текущей биграммы (tri[:2] == current_bigram). 
            # От ключей оставляем только последние слова (кандидаты на следующее слово в предложении) (tri[2]).
            # Представляем этот "подсловарь" в виде списка кортежей вида (слово, вероятность) (.items()).
            # Затем распаковываем этот список кортежей на два массива (слова-кандидаты и их вероятности) (zip)
            next_word = np.random.choice(candidate_words, p=word_probs)
            sent = sent + (next_word,)
            current_bigram = (current_bigram[1], next_word)
        sent = ' '.join(sent).replace('<start>', '').replace('<end>', '').strip()
        sent = glue_punctuation(sent)
        print(sent)

corpus = open('books.txt', encoding='utf-8').read()
sentences = [['<start>'] * 2 + filter_text(i) + ['<end>'] for i in sent_tokenize(corpus)]
train_sentences, test_sentences = sentences[:-50], sentences[-50:]
ngrams = {i: Counter() for i in range(1, 4)}
for sentence in train_sentences:
    for n in ngrams:
        ngrams[n].update(get_ngrams(sentence, n))
probabilities = {}
for trigram in ngrams[3]:
    probabilities[trigram] = ngrams[3][trigram] / ngrams[2][trigram[:2]]
# вероятности триграмм было решено хранить в словаре, так как на массив не хватало памяти

In [12]:
generate_sents(probabilities, 5)

Хороший ствол … разобрался я в ответ неопределенно повел рукой над столом и, наконец уже наговоришься, а вполне реальными патронами, который не брали, Сионист только назавтра должен был объяснять — было совершенно невозможно авиационное сообщение.
— не он, отрицательно помотав головой, шагнул в сторону и там же, какая дрянь водится уровнем ниже второй подвал есть.
— Рамзес придвинулся к Инге, — настороженно произнёс Чиба.
Коррупция в особо крупных отверстиях, расположенных по внешнюю сторону клинка, когда вертолет уйдет на очередной круг, будто оплакивая лучшего друга в Припяти с расчетом на месте.
Пасенки сидели в погребе больше ничего не расскажу!..


In [13]:
def perplexity(sent, ngr):
    probas = []
    for tri in get_ngrams(sent, 3):
        if tri in ngr[3] and tri[:2] in ngr[2]:
            probas.append(np.log(ngr[3][tri] / ngr[2][tri[:2]]))
        else:
            probas.append(np.log(0.00001))
    return np.exp(np.sum(probas)) ** (-1 / len(probas))

avg_perplexity = sum([perplexity(sent, ngrams) for sent in test_sentences]) / len(test_sentences)
print(f'Perplexity = {avg_perplexity}')

Perplexity = 13755.208879075031
